In [17]:
from pdf2txt import PDFProcessor
import pdfplumber
import json
from tqdm.auto import tqdm

d:\Python\Anaconda3\envs\NLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import os
try:
    print('1')
except:
    print('2')
else:
    file = './data/pdf/2021pdf年报/深圳能源：2021年年度报告.pdf'
    os.remove(file)

1


In [3]:
def extract_MDA(pdf_path):
    '''
    Extract the MDA parts from the pdf reports.
    '''
    with pdfplumber.open(pdf_path) as pdf:
        start_page = 0
        end_page = 0
        for i in range(10):  # 循环前10页，找到目录页
            content = pdf.pages[i].extract_text()
            if "......." in content:
                content = content.split('\n')
                #print(content)
                break
        
        for j in range(len(content)):
            if "与分析" in content[j] or "董事会" in content[j]:
                start_page = int(content[j][-2:])
                end_page = int(content[j+1][-2:])
                break
        
    return start_page, end_page

In [ ]:
file_path = './data/pdf/深圳能源：2021年年度报告.pdf'
start_page, end_page = extract_MDA(file_path)
processor = PDFProcessor(file_path)
processor.process_pdf(start_page, end_page)
processor.save_all_text(path='./data/txt/深圳能源：2021年年度报告.txt')

### clear txt

In [29]:
text_name = file_path.replace('pdf', 'txt')
content_clear = []
with open(text_name, 'r') as f:
    content = f.readlines()
    for i in range(len(content)):
        json_str = json.loads(content[i])
        if json_str['type'] == 'text' and '。' in json_str['inside']: # json_str['inside'][-1] == '。'
            sentences = json_str["inside"].split('。')
            if sentences[-1] == '':
                sentences.pop()            
            sentences = [item + '。\n' for item in sentences]
            content_clear.extend(sentences)

save_path = file_path.replace('pdf', 'txt_clear')
save_path = save_path[:-6] # delete the last "_clear" string
with open(save_path, 'w', encoding='utf-8') as file:
    file.writelines(content_clear)

In [ ]:
import json
file_path = './data/pdf/深圳能源：2021年年度报告.pdf'
text_name = file_path.replace('pdf','txt')
content = []
with open(text_name, 'r') as f:
    content = f.readlines()
json_str = json.loads(content[708])
json_str

{'page': 32,
 'allrow': 708,
 'type': 'text',
 'inside': '5．深化规范运作，提升治理水平新高度。一方面坚持做好上市工作运作基础工作，持续加强董事会自身建设，认真做好信息披露和投资者关系维护，向市场有效传递公司转型升级、低碳环保和高质量发展形象；另一方面深入推广实施大合规'}

In [53]:
text_name = file_path.replace('pdf', 'txt')
str_clear = ''
# content_clear = []
with open(text_name, 'r') as f:
    content = f.readlines()
    for i in range(len(content)):
        json_str = json.loads(content[i])
        if json_str['type'] == 'text' and '。' in json_str['inside']: # json_str['inside'][-1] == '。'
            str_clear = str_clear + json_str["inside"]
sentences = str_clear.split('。')
if sentences[-1] == '':
    sentences.pop()
content_clear = [item + '。\n' for item in sentences]

save_path = file_path.replace('pdf', 'txt_clear')
save_path = save_path[:-6] # delete the last "_clear" string
with open(save_path, 'w', encoding='utf-8') as file:
    file.writelines(content_clear)

In [ ]:
import json

txt_name = './data/txt/2021_002342_巨力索具.txt'
with open(txt_name, 'r', encoding='utf-8', errors='ignore') as f:
    content = f.readlines()
    print(content)
    for i in range(len(content)):
        json_str = json.loads(content[i])

### caculate

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from tqdm.auto import tqdm

dataset_name = "climatebert/climate_detection"
model_name = "climatebert/distilroberta-base-climate-detector"

# If you want to use your own data, simply load them as 🤗 Datasets dataset, see https://huggingface.co/docs/datasets/loading
dataset = datasets.load_dataset(dataset_name, split="test")

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

# See https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline
for out in tqdm(pipe(KeyDataset(dataset, "text"), padding=True, truncation=True)):
   print(out)

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, pipeline
from tqdm.auto import tqdm

tokenizer_climatebert = AutoTokenizer.from_pretrained("./model/distilroberta-base-climate-detector")
model_climatebert = AutoModelForSequenceClassification.from_pretrained("./model/distilroberta-base-climate-detector")

tokenizer_zh2en = AutoTokenizer.from_pretrained("./model/opus-mt-zh-en", use_fast=False)
model_zh2en = AutoModelForSeq2SeqLM.from_pretrained("./model/opus-mt-zh-en")

pipe1 = pipeline(task='translation', model=model_zh2en, tokenizer=tokenizer_zh2en, device=0, truncation=True)
pipe2 = pipeline(task='text-classification', model=model_climatebert, tokenizer=tokenizer_climatebert, device=0)

d:\Python\Anaconda3\envs\NLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Python\Anaconda3\envs\NLP\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [11]:
text_1 = '电化学能量存储装置是绿色、清洁能源的重要载体之一，在碳减排背景下广泛应用于绿色、清洁能源的存储、转换、使用，其重要性日益凸显。'
text_2 = '通过套期保值操作可以规避商品价格波动、汇率波动对公司造成的影响，有利于公司的正常经营，但同时也可能存在一定风险。' 

text = [text_1, text_2]
text

en_text = [m['translation_text'] for m in pipe1(text)]
pipe2(en_text)

[{'label': 'yes', 'score': 0.998123824596405},
 {'label': 'no', 'score': 0.9112256169319153}]

In [4]:
pipe1(text)

[{'translation_text': 'Electrochemical energy storage is one of the important carriers of green and clean energy, which is increasingly important in the context of carbon emission reductions and is widely applied to the storage, conversion and use of green and clean energy.'},
 {'translation_text': 'The impact on companies of commodity price fluctuations and exchange rate fluctuations can be avoided through hedging operations, which facilitates the normal operation of companies, but at the same time there may be some risk.'}]

In [4]:
total_sentences, yes_sentences

(67, 11)

In [2]:
import os

txt_path = './data/txt_clear/2021'
txt_list = os.listdir(txt_path)
# txt_path + '/' + txt_list[0]
#with open(txt_path+'/'+txt_list[0], 'r', encoding='utf-8') as f:
with open('./data/txt_clear/2019_002586_ST围海.txt', 'r', encoding='utf-8') as f:
    content = f.readlines()
    total_sentences = len(content)
    for i in range(total_sentences):
        content[i] = content[i].replace('\n', '')
    en_text = [m['translation_text'] for m in pipe1(content)]
    result = pipe2(en_text)
    yes_elements = [item for item in result if item['label'] == 'yes']
    yes_sentences = len(yes_elements)

Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForSeq2SeqLM
import torch
from tqdm.auto import tqdm


# model_path = 'C:/Users/Administrator/.cache/huggingface/hub'
model_path = './model'

tokenizer_climatebert = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-detector")
model_climatebert = AutoModelForSequenceClassification.from_pretrained("climatebert/distilroberta-base-climate-detector")

tokenizer_zh2en = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en", use_fast=False)
model_zh2en = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

def model_compute(sentence):
    # Translate Chinese sentence into English
    batch = tokenizer_zh2en([sentence], return_tensors="pt")
    generated_ids = model_zh2en.generate(**batch)
    tokenizer_zh2en.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Detect whether the sentence is related to climate risk
    inputs = tokenizer_climatebert(tokenizer_zh2en.batch_decode(generated_ids, skip_special_tokens=True)[0], return_tensors="pt")
    with torch.no_grad():
        logits = model_climatebert(**inputs).logits
    predicted_class_id = logits.argmax().item()
    return model_climatebert.config.id2label[predicted_class_id]

### logging

In [1]:
import logging

# 创建一个日志记录器
logger = logging.getLogger()

# 设置日志记录器的级别
logger.setLevel(logging.INFO)

# 创建一个文件处理器，用于将日志信息写入文件
file_handler = logging.FileHandler('output.log', mode='a')  # 'a' 表示追加模式

# 设置文件处理器的日志级别为ERROR，这样只有ERROR和CRITICAL级别的日志会被写入文件
file_handler.setLevel(logging.ERROR)

# 创建一个格式化器，用于定义日志的输出格式
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# 设置文件处理器的格式
file_handler.setFormatter(formatter)

# 将文件处理器添加到日志记录器
logger.addHandler(file_handler)

if __name__ == "__main__":
    for i in range(10):
        logger.info(f"{i}已保存.")
        logger.error(f"{i}报错.")

In [1]:
import logging

logging.basicConfig(filename='example.log', filemode='w', level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

if __name__ == "__main__":
    for i in range(10):
        logging.info(f"{i}已保存.")
        logging.error(f"{i}报错.")

### download_convert debug

In [ ]:
import os
import requests
import pdfplumber
import logging
from pdf2txt import PDFProcessor

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

flag_pdf = True
pdf_file_path = './data/pdf/2021pdf年报\\2021_000638_万方发展.pdf'
txt_file_path = './data/txt/2021txt年报\\2021_000638_万方发展.txt'
pdf_url = 'http://static.cninfo.com.cn/finalpage/2022-10-20/1214843056.PDF'
year = 2021
code = 638
name = '万方发展'

def download_pdf(pdf_url, pdf_file_path):
    try:
        with requests.get(pdf_url, stream=True, timeout=10) as r:
            r.raise_for_status()
            with open(pdf_file_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    except requests.exceptions.RequestException as e:
        logging.error(f"下载PDF文件失败：{e}")
        return False
    else:
        return True

In [ ]:
try:
    # 下载PDF文件
    if not os.path.exists(pdf_file_path):
        retry_count = 3
        while retry_count > 0:
            if download_pdf(pdf_url, pdf_file_path):
                break
            else:
                retry_count -= 1
        if retry_count == 0:
            logging.error(f"下载失败：{pdf_url}")
            #return
        
    # 提取MDA部分页码，转换PDF文件为TXT文件
    start_page, end_page = extract_MDA(pdf_file_path)
    processor = PDFProcessor(pdf_file_path)
    processor.process_pdf(start_page, end_page)
    processor.save_all_text(path=txt_file_path)
    processor.pdf.close() # 此处必须手动关闭pdf，因为PDFProcessor类定义中没有使用with

    logging.info(f"{txt_file_path} 已保存.")

except Exception as e:
    logging.error(f"处理 {year}_{code:06}_{name}时出错： {e}")
else:
    # 删除已转换的PDF文件，以节省空间
    if flag_pdf:
        os.remove(pdf_file_path)
        logging.info(f"{pdf_file_path} 已被删除.")

In [13]:
pdf_path = '1212711064.pdf'
extract_MDA(pdf_path)

ValueError: invalid literal for int() with base 10: '分析'

In [15]:
with pdfplumber.open(pdf_path) as pdf:
    start_page = 0
    end_page = 0
    for i in range(10):  # 循环前10页，找到目录页
        content = pdf.pages[i].extract_text()
        if "......." in content:
            content = content.split('\n')
            #print(content)
            break
    
    for j in range(len(content)):
        if "与分析" in content[j] or "董事会" in content[j]:
            start_page = int(content[j][-2:])
            end_page = int(content[j+1][-2:])
            break
        


ValueError: invalid literal for int() with base 10: '分析'

In [21]:
with pdfplumber.open(pdf_path) as pdf:
    start_page = 0
    end_page = 0
    content = pdf.pages[i].extract_text()

In [24]:
print(content)

深圳市中金岭南有色金属股份有限公司2021年年度报告全文
目录
第一节 重要提示、目录和释义
.........................................................................................................................................2
第二节 公司简介和主要财务指标
....................................................................................................................................6
第三节 管理层讨论与分析
.................................................................................................................................................. 11
第四节 公司治理
....................................................................................................................................................................... 47
第五节 环境和社会责任
....................................................................................................................................................... 67
第六节 重要事项
.................................................................................................................................................

In [10]:
import openpyxl
import os

year = 2021
txt_path = f'./data/pdf/{year}pdf年报'
txt_list = os.listdir(txt_path)
workbook = openpyxl.load_workbook(f'./data/excel/{year}年报remain.xlsx')
worksheet = workbook['Sheet1']

for txt_name in txt_list:
    year, code, name = txt_name.replace('.pdf', '').split('_')
    code = int(code)
    worksheet.append([code, name])

workbook.save(f'./data/excel/{year}年报remain.xlsx')
workbook.close()